# AI Implementation:

In [25]:
import random
from SourceCode import MancalaGameAI
import numpy as np
random.seed(50)

In [26]:
class RandomPlayer:
    def get_move(self, state):
        return state.random_move_generator()

In [27]:
#----------------- Minimax / AlphaBeta --------------------
    
def minimax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))

def alpha_beta_cutoff_search(state, game, d=4, cutoff_test=None, eval_fn=None):
    #Search game to determine best action; use alpha-beta pruning.
    #This version cuts off search and uses an evaluation function.
    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state: game.utility(state, player))
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [28]:
# AI player using Minimax
class MinimaxAI:
    def __init__(self):
        self.game_adapter = MancalaGameAI()

    def get_move(self, state):
        return minimax_decision(state, self.game_adapter)


# AI player using Alpha-Beta Pruning
class AlphaBetaAI:
    def __init__(self, depth=5):
        self.depth = depth
        self.game_adapter = MancalaGameAI()

    def get_move(self, state):
        return alpha_beta_cutoff_search(state, self.game_adapter, d=self.depth)

In [29]:
def play_game_AI(ai_player, opponent):
    game = MancalaGameAI()
    
    state = game.initial

    while not game.terminal_test(state):
        if game.to_move(state) == 1:
            move = ai_player.get_move(state)
        else:
            move = opponent.get_move(state)
        state = game.result(state, move)

    game.display(state)
    winner_score = game.utility(state, 1)
    if winner_score > 0:
        # print("AI wins")
        return 1
    elif winner_score < 0:
        # print("Opponent wins")
        return 2
    else:
        # print("Tie")
        return 0
    
    # Simulate 100 games
def sim_games_Minimax(num_games=100):
    ai_player = MinimaxAI()
    random_player = RandomPlayer()
    
    ai_wins = 0
    random_wins = 0
    ties = 0
    
    for i in range(num_games):
        result = play_game_AI(ai_player, random_player)
        
        if result == 1:
            ai_wins += 1
        elif result == 2:
            random_wins += 1
        else:
            ties += 1
    print("Minimax AI vs Random Player")
    print(f"AI Wins: {ai_wins}")
    print(f"Random Player Wins: {random_wins}")
    print(f"Ties: {ties}")

def sim_games_AlphaBeta(num_games=100):
    ai_player = AlphaBetaAI(depth=5)
    random_player = RandomPlayer()
    
    ai_wins = 0
    random_wins = 0
    ties = 0
    
    for i in range(num_games):
        result = play_game_AI(ai_player, random_player)
        
        if result == 1:
            ai_wins += 1
        elif result == 2:
            random_wins += 1
        else:
            ties += 1
    
    print("Alpha-Beta AI vs Random Player")
    print(f"AI Wins: {ai_wins}")
    print(f"Random Player Wins: {random_wins}")
    print(f"Ties: {ties}")

In [30]:
# Simulate 100 games with Minimax AI
sim_games_Minimax(num_games=100)

# Simulate 100 games with AlphaBeta AI
sim_games_AlphaBeta(num_games=100)

Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
It's a tie!
It's a tie!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
P

KeyboardInterrupt: 